<a href="https://colab.research.google.com/github/sarnika09-cell/MyBlog/blob/main/Photoshopie_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Step 1: Clone GitHub Repo (optional) ---
!git clone https://github.com/kying18/pyphotoshop.git
%cd pyphotoshop

# --- Step 2: Install Required Libraries ---
!pip install pillow numpy matplotlib ipywidgets --quiet

# --- Step 3: Imports ---
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os, shutil
from google.colab import files

# --- Step 4: Globals ---

_uploaded_img = None
_original_img = None
_current_img = None
_history = []

MAX_SIZE = 1000  # maximum width/height for uploaded images


# --- Step 5: Helper Functions ---
def resize_image(img):
    w, h = img.size
    if max(w, h) > MAX_SIZE:
        if w > h:
            new_w = MAX_SIZE
            new_h = int(h * MAX_SIZE / w)
        else:
            new_h = MAX_SIZE
            new_w = int(w * MAX_SIZE / h)
        img = img.resize((new_w, new_h), Image.LANCZOS)
    return img

def save_history():
    global _current_img, _history
    if _current_img is not None:
        _history.append(_current_img.copy())
        if len(_history) > 10:
            _history.pop(0)

def apply_filter(img, filter_type, brightness=1.5, contrast=1.3, blur=0):
    if filter_type == "Brightness":
        return ImageEnhance.Brightness(img).enhance(brightness)
    elif filter_type == "Contrast":
        return ImageEnhance.Contrast(img).enhance(contrast)
    elif filter_type == "Blur":
        return img.filter(ImageFilter.GaussianBlur(blur))
    elif filter_type == "Edges":
        return img.filter(ImageFilter.FIND_EDGES)
    elif filter_type == "Grayscale":
        return img.convert("L").convert("RGB")
    elif filter_type == "Invert":
        return Image.fromarray(255 - np.array(img))
    elif filter_type == "Warm":
        arr = np.array(img).astype(np.int16)
        arr[..., 0] = np.clip(arr[..., 0] + 40, 0, 255)
        return Image.fromarray(arr.astype(np.uint8))
    else:
        return img

# --- Step 6: Update Preview with Text and Crop ---
def update_preview(_=None):
    global _current_img, _original_img
    if _original_img is None:
        return
    img = _original_img.copy()
    img = apply_filter(img, filter_dropdown.value,
                       brightness_slider.value, contrast_slider.value, blur_slider.value)
    if rotate_slider.value != 0:
        img = img.rotate(rotate_slider.value)

    # Apply text
    if text_input.value.strip():
        draw = ImageDraw.Draw(img)
        img_w, img_h = img.size

        # Auto-scale text size
        font_size = max(int(img_h / 8), 24)
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
        except:
            font = ImageFont.load_default()

        try:
            bbox = draw.textbbox((0,0), text_input.value, font=font)
            text_w = bbox[2] - bbox[0]
        except AttributeError:
            text_w, _ = font.getsize(text_input.value)

        if text_w > img_w * 0.9:
            font_size = int(font_size * (img_w * 0.9) / text_w)
            try:
                font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
            except:
                font = ImageFont.load_default()

        x = min(max(0, text_x.value), img_w - 10)
        y = min(max(0, text_y.value), img_h - 10)
        draw.text((x, y), text_input.value, fill=text_color.value, font=font)

    _current_img = img
    with preview_output:
        clear_output(wait=True)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis("off")
        plt.show()

# --- Step 7: Upload Image ---
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
def on_upload(change):
    global _uploaded_img, _original_img, _current_img, _history
    for filename, file in upload_button.value.items():
        with open(filename, 'wb') as f:
            f.write(file['content'])
        img = Image.open(filename).convert("RGB")
        img = resize_image(img)
        _uploaded_img = img.copy()
        _original_img = img.copy()
        _current_img = img.copy()
        _history = []
        clear_output()
        print(f"✅ Image '{filename}' uploaded successfully!")
        display_ui()
        update_preview()
upload_button.observe(on_upload, names='value')
print("📸 Please upload your image:")
display(upload_button)

# --- Step 8: Widgets ---
filter_dropdown = widgets.Dropdown(
    options=["None","Brightness","Contrast","Blur","Edges","Grayscale","Invert","Warm"],
    description="Filter:"
)
brightness_slider = widgets.FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description='Brightness')
contrast_slider = widgets.FloatSlider(value=1.3, min=0.5, max=3.0, step=0.1, description='Contrast')
blur_slider = widgets.FloatSlider(value=0, min=0, max=5, step=0.5, description='Blur')
rotate_slider = widgets.IntSlider(value=0, min=0, max=360, step=10, description='Rotate')

text_input = widgets.Text(value='', description='Text:')
text_x = widgets.IntSlider(value=50, min=0, max=1000, step=10, description='X')
text_y = widgets.IntSlider(value=50, min=0, max=1000, step=10, description='Y')
text_color = widgets.ColorPicker(value='#ffffff', description='Color')
apply_text_button = widgets.Button(description="🖋️ Apply Text", button_style='info')

crop_x = widgets.IntSlider(value=0, min=0, max=1000, step=10, description='Crop X')
crop_y = widgets.IntSlider(value=0, min=0, max=1000, step=10, description='Crop Y')
crop_w = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Width')
crop_h = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Height')
apply_crop_button = widgets.Button(description="✂️ Crop", button_style='warning')

save_button = widgets.Button(description="💾 Save", button_style='success')
reset_button = widgets.Button(description="🔁 Reset", button_style='warning')
undo_button = widgets.Button(description="↩️ Undo", button_style='info')
delete_button = widgets.Button(description="🗑️ Delete", button_style='danger')
new_button = widgets.Button(description="🖼️ New Image", button_style='primary')
export_button = widgets.Button(description="📦 Export ZIP", button_style='primary')
preview_output = widgets.Output()

# --- Step 9: Button Handlers ---
def apply_text(b):
    save_history()
    update_preview()
    print(f"✅ Text '{text_input.value}' applied!")

def apply_crop(b):
    global _current_img
    if _current_img is None:
        print("⚠️ No image to crop!")
        return
    save_history()
    x, y, w, h = crop_x.value, crop_y.value, crop_w.value, crop_h.value
    _current_img = _current_img.crop((x, y, x + w, y + h))
    update_preview()
    print(f"✂️ Cropped region ({x},{y},{x+w},{y+h})")

def save_image(b):
    if _current_img is None:
        print("⚠️ No image to save!")
        return
    os.makedirs("outputs", exist_ok=True)
    path = f"outputs/edited_{len(os.listdir('outputs'))+1}.png"
    _current_img.save(path)
    print(f"✅ Saved: {path}")
    files.download(path)

def reset_image(b):
    global _current_img, _history
    _current_img = _original_img.copy()
    _history = []
    update_preview()
    print("🔁 Reset to original image!")

def undo_image(b):
    global _current_img, _history
    if len(_history) > 0:
        _current_img = _history.pop()
        update_preview()
        print("↩️ Undo last change")
    else:
        print("⚠️ No edits to undo")

def delete_image(b):
    global _uploaded_img, _original_img, _current_img, _history
    _uploaded_img = _original_img = _current_img = None
    _history = []
    with preview_output:
        clear_output()
    clear_output(wait=True)
    print("🗑️ Image deleted. Upload new image:")
    display(upload_button)

def new_image(b):
    clear_output()
    print("🖼️ Upload new image:")
    display(upload_button)

def export_zip(b):
    global _original_img
    if _original_img is None:
        print("⚠️ No image to export!")
        return
    output_dir = "outputs"
    os.makedirs(output_dir, exist_ok=True)
    filters = ["None","Brightness","Contrast","Blur","Edges","Grayscale","Invert","Warm"]
    export_path = os.path.join(output_dir, f"export_{len(os.listdir(output_dir))}")
    os.makedirs(export_path, exist_ok=True)
    for f in filters:
        filtered = apply_filter(_original_img, f)
        filtered.save(os.path.join(export_path, f"{f.lower()}.png"))
    shutil.make_archive(export_path, 'zip', export_path)
    print(f"📦 Exported all filters to {export_path}.zip")
    files.download(f"{export_path}.zip")

# --- Step 10: Bind Buttons ---
apply_text_button.on_click(apply_text)
apply_crop_button.on_click(apply_crop)
save_button.on_click(save_image)
reset_button.on_click(reset_image)
undo_button.on_click(undo_image)
delete_button.on_click(delete_image)
new_button.on_click(new_image)
export_button.on_click(export_zip)

# --- Step 11: Display UI ---
def display_ui():
    ui = widgets.VBox([
        widgets.HBox([filter_dropdown, brightness_slider, contrast_slider, blur_slider]),
        widgets.HBox([rotate_slider]),
        widgets.HBox([text_input, text_x, text_y, text_color, apply_text_button]),
        widgets.HBox([crop_x, crop_y, crop_w, crop_h, apply_crop_button]),
        widgets.HBox([save_button, reset_button, undo_button, delete_button, new_button, export_button]),
        preview_output
    ])
    for w in [filter_dropdown, brightness_slider, contrast_slider, blur_slider,
              rotate_slider, text_input, text_x, text_y, text_color,
              crop_x, crop_y, crop_w, crop_h]:
        w.observe(update_preview, names='value')
    display(ui)

✅ Image 'edited_1 (1).png' uploaded successfully!
